## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import callbacks
from sklearn.model_selection import ShuffleSplit, train_test_split 
from keras import losses, metrics 
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt  
from sklearn.metrics import confusion_matrix
from matplotlib.pyplot import figure

## Data Import

In [ ]:
Class_train = pd.read_csv("../Data/TrainandTestDataset.csv") 
Class_train_suffled = shuffle(Class_train, random_state=2)
Class_train_suffled.head()

## Feature Selection

In [11]:
features_col = ["Ia", "Ib", "Ic", "Va", "Vb", "Vc"] 
target_col = ["LG", "LL", "LLG", "LLL", "None"]
X = Class_train_suffled[features_col] 
Y = Class_train_suffled[target_col] 

## Feature Normalization

In [12]:
scalling = StandardScaler()
X_scalled = scalling.fit_transform(X, Y) 

## Dataset Splitting

In [13]:
x_train, x_val, y_train, y_val  = train_test_split(X_scalled, Y, test_size=0.2, random_state=10) 

## Model Design 

In [ ]:
Class_model = keras.Sequential([
    layers.Dense(60, activation='relu', input_shape=[6]),
    layers.BatchNormalization(),
    layers.Dense(100, activation='relu'), 
    layers.Dense(50, activation='relu'),   
    layers.Dense(5, activation = 'softmax')     
])

Class_model.compile(
    optimizer = 'adam', 
    loss = 'categorical_crossentropy',
    metrics = ['categorical_accuracy'], 
)

## Training

In [ ]:
history = Class_model.fit(
    x_train, y_train,
    validation_data = (x_val, y_val),
    batch_size = 600,  
    epochs = 1000
)

## Training History

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['categorical_accuracy', 'val_categorical_accuracy']].plot(); 
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Max train accuracy: {}".format(history_df['categorical_accuracy'].max()))
print("Max validation accuracy: {}".format(history_df['val_categorical_accuracy'].max()))

## Prediction on Validation Data

In [ ]:
y_pred = Class_model.predict(x_val)
for i in range(y_pred.shape[0]):
    a = y_pred[i, :]
    val = np.max(a)
    a[a != val] = 0
    a[a == val] = 1  
    y_pred[i, :] = a

In [ ]:
correct = 0
for i in range(y_pred.shape[0]): 
    if np.sum(y_val.iloc[i,:]==y_pred[i,:]) == 5:    
        correct = correct + 1
accuracy = correct / y_val.shape[0]
print(f"Validation Accuracy = {accuracy*100}%") 

## Confusion Matrix

In [ ]:
truelabel = []
predlabel = []

for i in range(y_val.shape[0]):
    if y_val.iloc[i, 0] == 1: 
        truelabel.append("LG")
    if y_val.iloc[i, 1] == 1:
        truelabel.append("LL")
    if y_val.iloc[i, 2] == 1:
        truelabel.append("LLG") 
    if y_val.iloc[i, 3] == 1:
        truelabel.append("LLL")
    if y_val.iloc[i, 4] == 1:
        truelabel.append("None") 
truelabel = np.array(truelabel) 

for i in range(y_pred.shape[0]):
    if y_pred[i, 0] == 1: 
        predlabel.append("LG")
    if y_pred[i, 1] == 1:
        predlabel.append("LL")
    if y_pred[i, 2] == 1:
        predlabel.append("LLG") 
    if y_pred[i, 3] == 1:
        predlabel.append("LLL")
    if y_pred[i, 4] == 1:
        predlabel.append("None") 

truelabel = np.array(truelabel) 
predlabel = np.array(predlabel) 

#Generate the confusion matrix
cf_matrix = confusion_matrix(truelabel, predlabel)

print(f"Confusion Matrix on Validation Data:\n{cf_matrix}")

In [ ]:
with plt.style.context('default'):
    
    figure(figsize=(7, 5), dpi=100)
    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]

    group_percentages = ["{0:.2%}".format(value) for value in
                        cf_matrix.flatten()/np.sum(cf_matrix)]

    labels = [f"{v1}\n{v2}\n" for v1, v2 in
            zip(group_counts,group_percentages)]

    labels = np.asarray(labels).reshape(5,5)

    ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='crest')

    ax.set_xlabel('\nPredicted Fault Type\n', fontsize = 12)
    ax.set_ylabel('\nActual Fault Type', fontsize = 13);

    ## Ticket labels - List must be in alphabetical order
    ax.xaxis.set_ticklabels(["LG", "LL", "LLG", "LLL", "None"])
    ax.yaxis.set_ticklabels(["LG", "LL", "LLG", "LLL", "None"])

    ## Display the visualization of the Confusion Matrix.
    plt.show()